# Image Provider 

In [ ]:
!pip install tensorflow==2.2.0

In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import os
import sys
BASE_PATH = os.path.dirname(os.getcwd())
os.chdir(BASE_PATH)

In [ ]:
from utils.datagenerator import TightFaceProvider
from utils.helper import show_images, cropper, glob_all_files, paths2numpy
import numpy as np 

# Train dataset

In [ ]:
fg_folder = './data/wally_face_tight_crop/'
bg_folder = './data/block_imgs/'
tfp = TightFaceProvider(fg_folder, bg_folder, batch_size=128)

In [ ]:
# 0번째 index의 batch 가져오기 
sample_imgs = tfp[0][0]
sample_labs = tfp[0][1]

# 이미지들이 pair 되었는지 확인합니다.  
show_images(sample_imgs, titles=sample_labs.tolist())

# Validation Dataset

In [ ]:
# validation images 을 생성합니다. 
# 정답 좌표는 아직 생성하지 않았습니다. 
paths = glob_all_files('./data/full_images_val/')

crop_imgs = []
crop_crds = []
imgs = paths2numpy(paths)
for img in imgs:
    cropped_images, cropped_coords = cropper(img, 10, 10, 34, 34)
    crop_imgs.append(cropped_images)
    crop_crds.append(cropped_coords)

val_imgs = np.concatenate(crop_imgs, axis=0)
print(val_imgs.shape)

# Simple Conv

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, ReLU
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [ ]:
K.clear_session()
inputs = Input(shape=(34, 34, 3), 
               name='inputs')

layer = inputs
# Convolution Layer
layer = Conv2D(filters=64, 
               kernel_size=3, 
               activation='relu')(layer)
layer = MaxPooling2D()(layer)

layer = Conv2D(filters=128, 
               kernel_size=3, 
               activation='relu')(layer)
layer = MaxPooling2D()(layer)

layer = Conv2D(filters=256, 
               kernel_size=3, 
               activation='relu')(layer)
layer = MaxPooling2D()(layer)

flat = Flatten()(layer)

# Fully Connected Layer
layer = flat
layer = Dense(units=256, activation='relu')(layer)
layer = Dense(units=256,activation='relu')(layer)

# Prediction
pred = Dense(1, activation='sigmoid')(layer)

# Model
model = Model(inputs, pred)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.save('./models/best_model2')

In [ ]:
model.summary()

In [ ]:
tfp[0][0].shape
tfp[0][1].shape

In [ ]:
# train 
for i in range(1):
    model.fit_generator(tfp, epochs=1)
    indices = (model.predict(val_imgs) > 0.5)[:, 0]
    show_images(val_imgs[indices])

# 전체 이미지에서 월리 찾아주기 

In [ ]:
import matplotlib.pyplot as plt 
import cv2

for img_ind, im in enumerate(imgs):
    val_imgs = crop_imgs[img_ind]
    val_crds = crop_crds[img_ind]
    
    indices = (model.predict(val_imgs) > 0.5)[:, 0]
    target_imgs = val_imgs[indices]
    target_crds = np.array(val_crds)[indices]
    
    fig = plt.figure(figsize=(20,20))
    for coords in target_crds:
        print(coords)
        im = cv2.rectangle(im, tuple(coords[:2]), tuple(coords[2:]), (255, 0, 0), 3)
    plt.imshow(im)
    plt.show()
